In [1]:
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [2]:
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [3]:
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [4]:
olddata = pd.read_csv("n_obj_df.csv")

In [5]:
olddata.head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [8]:
allregions = olddata.region_name.unique()

In [9]:
allregions

array(['Macclesfield', 'Starburst', 'Manhattan_Classic', 'Wellington',
       'Albany', 'Bilbao', 'Ithaca', 'Portsmouth', 'Pisaq',
       'Manhattan_Frontinella', 'BuenosAires', 'Inca_City_Ridges',
       'Inca_City', 'Giza', 'Potsdam', 'Troy', 'Oswego_Edge', 'Halifax',
       'Caterpillar', 'Rochester', 'Manhattan_Cracks', 'Schenectady',
       'Binghamton', 'Atka', 'Cortland', 'Geneseo'], dtype=object)

In [10]:
for region in allregions:
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['region_name'] = region
        d['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d['Obsid_Area'] = area
        d['polygon_shape'] = x
        bucket.append(d)
    results = pd.DataFrame(bucket)
    
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['obsid'] = obs #we dont want obs, want obs used in combo
        x = get_outer_polygon_for_obsid(obs)
        d['obs_poly'] = x
        bucket.append(d)
    testresult = pd.DataFrame(bucket)
    
    df1 = testresult
    
    df2 = interchecker(testresult.obs_poly)
    
    Testregion_inter = interchecker(results.polygon_shape) #all combos of results
    
    bucket2 = []
    for ele in Testregion_inter.index:
        polylist = Testregion_inter.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.obsid[df1['obs_poly'] == poly].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    Testregion_inter['obsids_used'] = np.array(bucket2)

    obsdf = Testregion_inter
    
    bucket = []
    for i in range(len(obsdf.elements_used)): #change to range(len(obsdf.elements_used)) after we check
        #print(len(obsdf.elements_used[i])) #check it's what we want.
        obsdf.at[i, 'number_hirise'] = int(len(obsdf.elements_used[i]))
        #even calling it int(len) still gives float on df, but whatever.
        
        
    bucket2 = []
    for obslist in obsdf.obsids_used:
        bucket = []
        for obs in obslist:
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    obsdf['Ls_used'] = np.array(bucket2)
    obsdf['region_name'] = region

    obsdf.to_csv(f'{region}_duplicate_test_combodf.csv', index = False)

TypeError: float() argument must be a string or a number, not 'Point'